In [12]:
# to create data tables
import geopandas as gpd
import pandas as pd

# to transform UTM to WGS84
from shapely.geometry import Polygon
from pyproj import Proj, transform

# to create the interactive map
import folium
import json

# to parse the KML files and plot the polygons
from fastkml import kml
from shapely.geometry import Polygon, Point, LineString

In [13]:
xl = pd.ExcelFile('SWB - Installation Phasing.xlsx')
df = xl.parse(xl.sheet_names[0])
df = df.dropna(subset=['Easting', 'Northing'])
df.head()

,#,Tag,Country,Site,Plot,Label,Date,Type,Design,Fill,...,Easting,Northing,NW-E,NW-N,NE-E,NE-N,SE-E,SE-N,SW-E,SW-N
0,1.0,Channel 1,NaN,SWB,NaN,NaN,2023-09-01 00:00:00,Channel,-,Red,...,510750.0,7054150.0,510700.0,7054650.0,510800.0,7054650.0,510800.0,7053650.0,510700.0,7053650.0
1,2.0,Channel 2,NaN,SWB,NaN,NaN,2023-09-01 00:00:00,Channel,-,Red,...,510250.0,7054700.0,509300.0,7054750.0,511200.0,7054750.0,511200.0,7054650.0,509300.0,7054650.0
2,3.0,Channel 3,NaN,SWB,NaN,NaN,2023-09-01 00:00:00,Channel,-,Red,...,510050.0,7055200.0,510000.0,7055600.0,510100.0,7055600.0,510100.0,7054800.0,510000.0,7054800.0
3,4.0,NAM-SWB-AAA,NAM,SWB,AAA,NaN,01/09/2023,BB+NM,Shallow,Green,...,510470.0,7053790.0,510465.0,7053815.0,510475.0,7053815.0,510475.0,7053765.0,510465.0,7053765.0
4,5.0,NAM-SWB-AAB,NAM,SWB,AAB,NaN,01/09/2023,BB+NM,Shallow,Green,...,510450.0,7053720.0,510429.0,7053765.0,510471.0,7053765.0,510471.0,7053675.0,510429.0,7053675.0


In [14]:
# Define the Proj objects for the transformations
utm33 = Proj(proj='utm', zone=33, ellps='WGS84', preserve_units=False)
wgs84 = Proj(proj='latlong', datum='WGS84')

In [15]:
def utm_to_latlon(row, easting_col, northing_col):
    # convert UTM to lat/lon
    northing = 10000000 - row[northing_col] # adjust for southern hemisphere
    lon, lat = transform(utm33, wgs84, row[easting_col], northing)
    return pd.Series({'Latitude': -lat, 'Longitude': lon}) # latitude is negative in the southern hemisphere


# apply the function to each pair of columns
coordinate_columns = [('Easting', 'Northing'), 
                      ('NW-E', 'NW-N'), 
                      ('NE-E', 'NE-N'), 
                      ('SE-E', 'SE-N'), 
                      ('SW-E', 'SW-N')]

for easting_col, northing_col in coordinate_columns:
    df[[f'{easting_col}_Latitude', f'{easting_col}_Longitude']] = df.apply(lambda row: utm_to_latlon(row, easting_col, northing_col), axis=1)

C:\Users\Hugo Stam\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  after removing the cwd from sys.path.


In [16]:
# Create a map centered at an arbitrary location
m = folium.Map(location=[df['Easting_Latitude'].mean(), df['Easting_Longitude'].mean()], zoom_start=14)

# Add a color-coded polygon for each point
for idx, row in df.iterrows():
    polygon_points = [
        [row['NW-E_Latitude'], row['NW-E_Longitude']],
        [row['NE-E_Latitude'], row['NE-E_Longitude']],
        [row['SE-E_Latitude'], row['SE-E_Longitude']],
        [row['SW-E_Latitude'], row['SW-E_Longitude']]
    ]
    folium.Polygon(polygon_points, 
                   color=row['Colour'], 
                   fill_color=row['Colour'], 
                   fill_opacity=0.5).add_to(m)
    
 # Add smaller circle markers at each corner
    for lat, lon in polygon_points:
        folium.CircleMarker([lat, lon], radius=3, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(m)

# Display the map
# m

KeyError: 'Colour'

In [ ]:
print("Number of polygons:", len(df))

In [ ]:
def plot_on_map(df, start_date=None, end_date=None, start_row=None, end_row=None):
    # Filter based on date if specified
    if start_date and end_date:
        df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    # Subset based on row numbers if specified
    if start_row is not None or end_row is not None:
        df = df.iloc[start_row:end_row]

    # Create a map centered at an arbitrary location
    m = folium.Map(location=[df['Easting_Latitude'].mean(), df['Easting_Longitude'].mean()], zoom_start=13)

    # Add a color-coded polygon and smaller circle markers for each point in the subset
    for idx, row in df.iterrows():
        polygon_points = [
            [row['NW-E_Latitude'], row['NW-E_Longitude']],
            [row['NE-E_Latitude'], row['NE-E_Longitude']],
            [row['SE-E_Latitude'], row['SE-E_Longitude']],
            [row['SW-E_Latitude'], row['SW-E_Longitude']]
        ]
        folium.Polygon(polygon_points, 
                       color=row['Colour'], 
                       fill_color=row['Colour'], 
                       fill_opacity=0.5).add_to(m)

        # Add smaller circle markers at each corner with the same color as the polygon
        for lat, lon in polygon_points:
            folium.CircleMarker([lat, lon], 
                                radius=3, 
                                color=row['Colour'], 
                                fill=True, 
                                fill_color=row['Colour'], 
                                fill_opacity=1).add_to(m)

    # Display the map
    return m

# Ensure the 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Call the function specifying the date range for Sep-Nov
start_date='2022-07-01'
end_date='2024-11-30'

map_object = plot_on_map(df, start_date = start_date, end_date = end_date)
map_object

In [ ]:
start_date='2023-08-30'
end_date='2023-11-30'
map_object = plot_on_map(df, start_date = start_date, end_date = end_date)

map_object